In [ ]:
import cPickle as cpkl
import sys, logging, os
from sklearn import metrics 
slim_path = '/root/workspace/workspace/ROP_PAPER/slim'
if slim_path not in sys.path:
    sys.path.append(slim_path)
from nets.inception_v2 import *
from nets.inception_utils import *

import tensorflow as tf
import numpy as np
import tensorflow.contrib.slim as slim

from sacred import Experiment
from data import *
from InceptionV2 import *


In [ ]:
data_path = '/mnt/data/lab-data/medical/ROP_data_list/20170828/verified_data_list(stage2,3)_consistent.txt.pkl'
#data_path =  '/mnt/data/lab-data/medical/ROP_data_list/20170727/verified_data_list(normal,rop).txt.pkl'
network = 'InceptionV2'
gpu = 1
num_classes = 2
batch_size = 8
num_epochs = 200
imgs_per_sample = 16
disp_batches = 10
lr = 1.0
optimizer = 'sgd'
network_prefix = 'G-NET'
save_checkpoint_path = './logs'
drop_o=0.5 
init_scale = 0.01
img_height = 240
img_width = 320
img_channels = 3
checkpoint_exclude_scopes = [network + '/logits', network + '/AuxLogits']
checkpoint_path = '../slim/checkpoints/inception_v2.ckpt'
data_balance = True
fold = 4
seed = 275381447
log_path = '%s/%s-%s/%d' % (save_checkpoint_path, network_prefix, network, seed)

class Conf(object):
    def __init__(self):
        pass
net_config = Conf()
net_config.num_classes = num_classes
net_config.imgs_per_sample = imgs_per_sample
net_config.img_height = img_height
net_config.img_width = img_width
net_config.init_scale = init_scale
net_config.img_channels = img_channels
net_config.lr = lr
epoch = 100

In [8]:
#data_path = '/mnt/data/lab-data/medical/ROP_data_list/20170828/verified_data_list(stage2,3)_consistent.txt.pkl'
data_path =  '/mnt/data/lab-data/medical/ROP_data_list/20170727/verified_data_list(normal,rop).txt.pkl'
network = 'InceptionV2'
gpu = 1
num_classes = 2
batch_size = 8
num_epochs = 200
imgs_per_sample = 16
disp_batches = 10
lr = 1.0
optimizer = 'sgd'
network_prefix = 'I-NET'
save_checkpoint_path = './logs'
drop_o=0.5 
init_scale = 0.01
img_height = 240
img_width = 320
img_channels = 3
checkpoint_exclude_scopes = [network + '/logits', network + '/AuxLogits']
checkpoint_path = '../slim/checkpoints/inception_v2.ckpt'
data_balance = True
fold = 4
seed = 837835977
log_path = '%s/%s-%s/%d' % (save_checkpoint_path, network_prefix, network, seed)

class Conf(object):
    def __init__(self):
        pass
net_config = Conf()
net_config.num_classes = num_classes
net_config.imgs_per_sample = imgs_per_sample
net_config.img_height = img_height
net_config.img_width = img_width
net_config.init_scale = init_scale
net_config.img_channels = img_channels
net_config.lr = lr
epoch = 6

In [3]:
## load pretrain model callback
def get_init_fn(checkpoint_path, checkpoint_exclude_scopes=[]):
    exclusions = [scope.strip() for scope in checkpoint_exclude_scopes]
    variables_to_restore = []
    for var in slim.get_model_variables():#tf.trainable_variables():#
        excluded = False
        for exclusion in exclusions:
            if var.op.name.startswith(exclusion):
                excluded = True
                break
        if not excluded:
            variables_to_restore.append(var)
    return slim.assign_from_checkpoint_fn(checkpoint_path, variables_to_restore, ignore_missing_vars=True), variables_to_restore


In [4]:
arg_scope = inception_arg_scope(batch_norm_decay=0.9)
img_aug = ImageAug(img_height, img_width, img_channels)
train_datas, train_labels, test_datas, test_labels = cpkl.load(open(log_path+'/data.pkl',"rb"))

test_data = RetinaDataIter(test_datas, imgs_per_sample, test_labels, batch_size, 
                           img_height, img_width, img_channels,
                           image_aug=img_aug)

In [5]:
def test_epoch(sess, model, test_data): 
    batch_idx = 0
    accs = 0.
    costs = 0.
    test_data.reset()
    preds = []
    labels = []
    for input_data, label in test_data:
        feed_dicts = {model.input_data:input_data, model.label:label, model.dropout_keep_prob:1.}
        acc, pred, cost = sess.run([model.accuracy, model.preds, model.cost,], feed_dicts)
        preds.append(pred)
        labels.append(label)
        batch_idx += 1
        print batch_idx
    log_msg = check_metric(preds, labels)
    return log_msg
def check_metric(preds, labels):
    for i, (pred, label) in enumerate(zip(preds, labels)):
        if i == 0:
            p_l = pred
            l_l = label
        else:
            p_l = np.concatenate([p_l, pred])
            l_l = np.concatenate([l_l, label])
    idx_l_l = np.argmax(l_l, axis=1)
    idx_p_l = np.argmax(p_l, axis=1)
    TP = len([1 for (p, l) in zip(idx_p_l, idx_l_l) if p==1 and l==1])
    FP = len([1 for (p, l) in zip(idx_p_l, idx_l_l) if p==1 and l==0])
    TN = len([1 for (p, l) in zip(idx_p_l, idx_l_l) if p==0 and l==0])
    FN = len([1 for (p, l) in zip(idx_p_l, idx_l_l) if p==0 and l==1])
    precision_score = metrics.precision_score(idx_l_l, idx_p_l)
    recall_score = metrics.recall_score(idx_l_l, idx_p_l)
    accuracy_score = metrics.accuracy_score(idx_l_l, idx_p_l)
    fpr, tpr, thresholds = metrics.roc_curve(idx_l_l, p_l[:,1], pos_label=1)
    acu = metrics.auc(fpr, tpr)
    log_msg = 'precision_score:%.5f, recall_score:%.5f, accuracy_score:%.5f, acu:%.5f' % (precision_score,recall_score,accuracy_score,acu)
    return log_msg, (fpr, tpr), (TP, FP, TN, FN)

In [9]:
initializer = tf.random_uniform_initializer(-net_config.init_scale, net_config.init_scale)
gpu_config = tf.ConfigProto()
gpu_config.gpu_options.allow_growth = True
gpu_config.gpu_options.visible_device_list = str(0)
gpu_config.gpu_options.per_process_gpu_memory_fraction = 0.95
with tf.Graph().as_default(), tf.Session(config=gpu_config) as sess:
    tf.logging.set_verbosity(tf.logging.INFO)
    with slim.arg_scope(arg_scope):
        with slim.arg_scope([slim.batch_norm], is_training=True):
            with tf.name_scope('train'):
                with tf.variable_scope(network, reuse=None, initializer = initializer) as scope:
                    train_model = globals()[network](is_training=True, config=net_config, scope=scope)
    with slim.arg_scope(arg_scope):
        with slim.arg_scope([slim.batch_norm], is_training=False):
            with tf.name_scope('test'):
                with tf.variable_scope(network, reuse=True, initializer = initializer) as scope:
                    test_model = globals()[network](is_training=False, config=net_config, scope=scope)
                    
    print '[ Loading checkpoint ... ]'
    weight_path = "%s/epoch-%d.ckpt" % (log_path, epoch)
    init_fn, restore_vars = get_init_fn(weight_path, [])
    init_fn(sess)

    # init left variables in model
    print 'init left...'
    uninitialized_vars = set(tf.global_variables()) - set(restore_vars)#set(tf.trainable_variables()) - set(restore_vars)
    #print uninitialized_vars
    sess.run(tf.variables_initializer(uninitialized_vars))
    log_msg,(fpr, tpr),(TP, FP, TN, FN) = test_epoch(sess, test_model, test_data,)
    print 'Epoch [%d] test on test %s' % (epoch, log_msg)

(u'InceptionV2/Conv2d_1a_7x7/depthwise_weights:0', [7, 7, 3, 8])
(u'InceptionV2/Conv2d_1a_7x7/pointwise_weights:0', [1, 1, 24, 64])
(u'InceptionV2/Conv2d_1a_7x7/biases:0', [64])
(u'InceptionV2/Conv2d_2b_1x1/weights:0', [1, 1, 64, 64])
(u'InceptionV2/Conv2d_2b_1x1/BatchNorm/beta:0', [64])
(u'InceptionV2/Conv2d_2c_3x3/weights:0', [3, 3, 64, 192])
(u'InceptionV2/Conv2d_2c_3x3/BatchNorm/beta:0', [192])
(u'InceptionV2/Mixed_3b/Branch_0/Conv2d_0a_1x1/weights:0', [1, 1, 192, 64])
(u'InceptionV2/Mixed_3b/Branch_0/Conv2d_0a_1x1/BatchNorm/beta:0', [64])
(u'InceptionV2/Mixed_3b/Branch_1/Conv2d_0a_1x1/weights:0', [1, 1, 192, 64])
(u'InceptionV2/Mixed_3b/Branch_1/Conv2d_0a_1x1/BatchNorm/beta:0', [64])
(u'InceptionV2/Mixed_3b/Branch_1/Conv2d_0b_3x3/weights:0', [3, 3, 64, 64])
(u'InceptionV2/Mixed_3b/Branch_1/Conv2d_0b_3x3/BatchNorm/beta:0', [64])
(u'InceptionV2/Mixed_3b/Branch_2/Conv2d_0a_1x1/weights:0', [1, 1, 192, 64])
(u'InceptionV2/Mixed_3b/Branch_2/Conv2d_0a_1x1/BatchNorm/beta:0', [64])
(u'Ince

[ Loading checkpoint ... ]
INFO:tensorflow:Restoring parameters from ./logs/I-NET-InceptionV2/837835977/epoch-6.ckpt
init left...
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
Epoch [6] test on test precision_score:0.99306, recall_score:0.96622, accuracy_score:0.97973, acu:0.99493


In [10]:
(TP, FP, TN, FN)

(143, 1, 147, 5)

In [ ]:
from scipy import interp
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

matplotlib.rcParams['font.sans-serif'] = ['Times New Roman', 'serif', 'sans-serif', 'cursive', 'fantasy', 'monospace']
matplotlib.rcParams['font.size'] = '12'

mean_tpr = 0.0    
mean_fpr = np.linspace(0, 1, 100)  
mean_tpr += interp(mean_fpr, fpr, tpr)
print mean_tpr,mean_fpr
mean_tpr[0] = 0.0

fig = plt.figure()
plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Luck')  
plt.plot(fpr, tpr,'--')
plt.xlim([-0.05, 1.05])  
plt.ylim([-0.05, 1.05])  
plt.xlabel('False Positive Rate')  
plt.ylabel('True Positive Rate')  
#plt.title('Receiver operating characteristic example')  
#plt.legend(loc="lower right") 
fig.savefig('roc-I-Net')
plt.show()  